In [12]:
import module.TorchDataset_aug as td
import platform
import importlib

importlib.reload(td) # 클래스 수정 시 import 리로드

# os에 따라 다르게 적용 (코랩은 리눅스)
current_os = platform.system()
sep = "\\" if current_os == "Windows" else "/" 

img_size = 224
root_test = ""
etc_test = ""

# TestSet
if current_os == "Windows":
    root_test = f"E:{sep}02.공부{sep}02.코딩{sep}01.Python{sep}01.Alphaco{sep}02.코드{sep}03.프로젝트{sep}03.이미지분류{sep}01.Data{sep}fire{sep}test{sep}" # 테스트 루트 폴더
    etc_test = f"E:{sep}02.공부{sep}02.코딩{sep}01.Python{sep}01.Alphaco{sep}02.코드{sep}03.프로젝트{sep}03.이미지분류{sep}01.Data{sep}fire{sep}etc{sep}" # 테스트 루트 폴더
else:
    root_test = f"01.Data{sep}fire{sep}test{sep}"
    etc_test = f"etc{sep}"

extension = "jpg" # 찾을 확장자
test_dataset = td.TorchDataset(root_test, sep, extension, img_size, "test", False)
etc_dataset = td.TorchDataset(etc_test, sep, extension, img_size, "test", True)


# Hidden set
# root_test = f"E:{sep}02.공부{sep}02.코딩{sep}01.Python{sep}01.Alphaco{sep}02.코드{sep}03.프로젝트{sep}03.이미지분류{sep}01.Data{sep}fire{sep}etc{sep}" # 테스트 루트 폴더
# extension = "jpg" # 찾을 확장자
# test_dataset = td.TorchDataset(root_test, sep, extension, img_size, "test", False)

print(len(test_dataset), len(etc_dataset))
print(test_dataset.get_label_dict())

6299 96
{'nowildfire': 0, 'wildfire': 1}


In [13]:
import module.TorchDatasetCNN as tdCNN
importlib.reload(tdCNN)

cnn_runner = tdCNN.TorchDatasetCNN()
batch_size = 10

test_dataloader = cnn_runner.get_dataloader(test_dataset, batch_size, False, False)
etc_dataloader = cnn_runner.get_dataloader(etc_dataset, batch_size, False, False)
len(test_dataloader), len(etc_dataloader)

(630, 10)

In [3]:
######################## Ensenble CNN #####################
###########################################################
###########################################################
###########################################################

In [14]:
import torch.nn as nn
from torchvision import models

device = cnn_runner.get_device()

custom_save_path = ""
resnet50_layer4_save_path = ""
resnet50_full_save_path = ""
googlenet_save_path = ""


if current_os == "Windows": # 로컬 저장용
    custom_save_path = f"save_model{sep}03.Torch_CNN{sep}02.9_layer_Custom_CNN{sep}fire_custom_9_CNN.pth"
    resnet50_layer4_save_path = f"save_model{sep}03.Torch_CNN{sep}05.resnet50_Layer4_Finetune{sep}resnet50_Layer4_Finetune.pth"
    resnet50_full_save_path = f"save_model{sep}03.Torch_CNN{sep}04.resnet50_Full_Finetune{sep}resnet50_Full_Finetune.pth"
    googlenet_save_path = f"save_model{sep}03.Torch_CNN{sep}06.googlenet_Finetune{sep}googlenet_Finetune.pth"
    
else:
    custom_save_path = f"save_model{sep}fire_custom_9_CNN.pth"
    resnet50_layer4_save_path = f"save_model{sep}resnet50_Layer4_Finetune.pth" # 코랩 저장용
    resnet50_full_save_path = f"save_model{sep}resnet50_Full_Finetune.pth"
    googlenet_save_path = f"save_model{sep}googlenet_Finetune.pth"

In [15]:
import module.CNN as CNN
importlib.reload(CNN)

# Custom CNN 로드
custom_model = CNN.CNN(img_size).to(device)
custom_model = cnn_runner.load_model(custom_model, custom_save_path, device)
custom_model.to(device)

# ResNet layer4 Finetune CNN 로드
import torch.nn as nn
from torchvision import models
resnet_model = models.resnet50().to(device)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2) # fc layer 수정
resnet_model = cnn_runner.load_model(resnet_model, resnet50_layer4_save_path, device)
resnet_model = resnet_model.to(device)

# ResNet full Finetune CNN 로드
resnet_full_model = models.resnet50().to(device)
resnet_full_model.fc = nn.Linear(resnet_full_model.fc.in_features, 2) # fc layer 수정
resnet_full_model = cnn_runner.load_model(resnet_full_model, resnet50_full_save_path, device)
resnet_full_model = resnet_full_model.to(device)


# Googlenet inseption5b finetuen CNN 로드
googlenet_finetune = models.googlenet(weights='DEFAULT').to(device)
googlenet_finetune.fc = nn.Linear(googlenet_finetune.fc.in_features, 2) # fc layer 수정
googlenet_finetune = cnn_runner.load_model(googlenet_finetune, googlenet_save_path, device)
googlenet_finetune = googlenet_finetune.to(device)

In [ ]:
# 앙상블 모델 테스트
# kind = "custom_9, resnet50_4_F"
# cnn_runner.test_ensenble_model([custom_model, resnet_model], test_dataloader, device, kind)

In [11]:
# 앙상블 모델 테스트
# kind = "resnet50_4_F, resnet50_Full, custom_9"
# cnn_runner.test_ensenble_model([custom_model, resnet_model, resnet_full_model], test_dataloader, device, kind)

100%|██████████| 315/315 [25:29<00:00,  4.85s/it]

resnet50_4_F, resnet50_Full
일치 : 6253 / 6299, 정확도 : 99.27
리콜 = tp : 3463, fn : 16 / racall : 99.54


In [12]:
# 앙상블 모델 테스트
# kind = "custom_9, resnet50_Full"
# cnn_runner.test_ensenble_model([custom_model, resnet_full_model], test_dataloader, device, kind)

100%|██████████| 315/315 [14:19<00:00,  2.73s/it]

custom_9, resnet50_Full
일치 : 6181 / 6299, 정확도 : 98.13
리콜 = tp : 3442, fn : 37 / racall : 98.94


In [ ]:
# 앙상블 모델 테스트
kind = "resnet50_4_F, resnet50_Full"
cnn_runner.test_ensenble_model([resnet_model, resnet_full_model], test_dataloader, device, kind)

In [8]:
# 앙상블 모델 테스트
kind = "resnet50_4_F, googlenet_5b"
cnn_runner.test_ensenble_model([googlenet_finetune, resnet_model], test_dataloader, device, kind)

100%|██████████| 315/315 [21:12<00:00,  4.04s/it]

resnet50_4_F, googlenet_5b
일치 : 6258 / 6299, 정확도 : 99.35
리콜 = tp : 3464, fn : 15 / racall : 99.57


In [17]:
# 모델 테스트
kind = "resnet50_4_F, googlenet_5b"
cnn_runner.test_ensenble_model([googlenet_finetune], etc_dataloader, device, kind)

100%|██████████| 10/10 [00:08<00:00,  1.16it/s]

resnet50_4_F, googlenet_5b
일치 : 62 / 96, 정확도 : 64.58
리콜 = tp : 34, fn : 12 / racall : 73.91


In [29]:
temp_path = "C:\\Users\\HS\\OneDrive\\바탕 화면\\"
temp_dataset = td.TorchDataset(temp_path, sep, extension, img_size, "test", False)

temp_dataloader = cnn_runner.get_dataloader(temp_dataset, batch_size, False, False)
len(temp_dataset), len(temp_dataloader)

(1, 1)

In [ ]:
# 앙상블 모델 테스트
kind = "resnet50_4_F, resnet50_Full"
cnn_runner.test_ensenble_model([resnet_model, resnet_full_model], temp_dataloader, device, kind)